This notebook contains code for translating sentence-pairs to create initial datasets

In [1]:
!git clone https://github.com/AI4Bharat/indicTrans.git
%cd indicTrans
# clone requirements repositories
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone https://github.com/rsennrich/subword-nmt.git
%cd ..

In [2]:
# Install the necessary libraries
!pip install sacremoses pandas mock sacrebleu tensorboardX pyarrow indic-nlp-library
!pip install mosestokenizer subword-nmt
# Install fairseq from source
!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
# !git checkout da9eaba12d82b9bfc1442f0e2c6fc1b895f4d35d
!pip install ./
!pip install xformers
%cd ..

In [3]:
!ls

Data	      extracting_data.py   requirements.txt   train.tsv
dev.tsv       fairseq		   sentences_in.txt   translate_notebook.ipynb
en2indic.zip  indic_nlp_resources  sentences_out.txt
en-indic      indicTrans	   test.tsv


In [1]:
# add fairseq folder to python path
import os
os.environ['PYTHONPATH'] += "fairseq/"
# sanity check to see if fairseq is installed
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils

Blocksparse is not available: the current GPU does not expose Tensor cores


In [2]:
# # download the indictrans model

# downloading the en-indic model
# !wget https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/en2indic.zip
# !unzip en2indic.zip

# # # downloading the indic-indic model
# # !wget https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/m2m.zip
# # !unzip m2m.zip

%cd indicTrans

/home/neham/vibhhu/project_final/indicTrans


In [3]:
from indicTrans.inference.engine import Model

en2indic_model = Model(expdir='../en-indic')

Initializing vocab and bpe
Initializing model for translation


2023-05-02 21:21:07 | INFO | fairseq.tasks.translation | [SRC] dictionary: 32104 types
2023-05-02 21:21:07 | INFO | fairseq.tasks.translation | [TGT] dictionary: 35888 types


In [7]:
!ls

AI4B_Demo				interface
api					joint_score.sh
apply_bpe_traindevtest_notag.sh		joint_translate.sh
apply_single_bpe_traindevtest_notag.sh	learn_bpe.sh
binarize_training_exp.sh		learn_single_bpe.sh
compute_bleu.sh				legacy
indic_nlp_library			LICENSE
indic_nlp_resources			model_configs
indictrans_fairseq_inference.ipynb	prepare_data_joint_training.sh
indicTrans_Finetuning.ipynb		prepare_data.sh
indicTrans_hosted_api_inference.ipynb	README.md
indicTrans_python_interface.ipynb	sample_images
IndicTrans_training.ipynb		scripts
inference				subword-nmt


In [4]:
en_sents=[]
with open('../nonparaphrases_out.txt') as f:
    en_sents = f.readlines()

In [3]:
import torch

In [4]:
torch.cuda.is_available()

True

In [5]:
a=en2indic_model.batch_translate(en_sents[:4000], 'en', 'hi')

100%|██████████| 4000/4000 [00:01<00:00, 3981.69it/s]
2023-05-02 21:21:22 | INFO | fairseq.tasks.fairseq_task | can_reuse_epoch_itr = True
2023-05-02 21:21:22 | INFO | fairseq.tasks.fairseq_task | reuse_dataloader = True
2023-05-02 21:21:22 | INFO | fairseq.tasks.fairseq_task | rebuild_batches = False
2023-05-02 21:21:22 | INFO | fairseq.tasks.fairseq_task | creating new batches for epoch 1


In [7]:
from tqdm import tqdm

In [17]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-hi")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-hi")
model.to(0)

# Example English sentences
english_sentences = en_sents

# Tokenize the English sentences
inputs = tokenizer(english_sentences, padding=True, truncation=True, return_tensors="pt")
inputs.to(0)
# Set the batch size
batch_size = 8

# Generate the Hindi translations in batches
hindi_sentences = []
for i in tqdm(range(0, len(inputs.input_ids), batch_size)):
    input_ids_batch = inputs.input_ids[i:i+batch_size]
    attention_mask_batch = inputs.attention_mask[i:i+batch_size]
    outputs = model.generate(input_ids_batch, attention_mask=attention_mask_batch, 
                             max_length=40, num_beams=4, early_stopping=True)
    for output in outputs:
        hindi_sentence = tokenizer.decode(output, skip_special_tokens=True)
        hindi_sentences.append(hindi_sentence)

print("Done")


100%|██████████| 3447/3447 [23:59<00:00,  2.39it/s]

Done


'बीबाजी ने अपने अलग विचारों को स्वाग काल की किताब के माध्यम से फैला दिया, कजेल कोआ, आदि.'

In [6]:
file = open('hindi_output_nonpara.txt','w')
for item in a:
	file.write(item+"\n")
file.close()

In [ ]:

# ta_paragraph = """இத்தொற்றுநோய் உலகளாவிய சமூக மற்றும் பொருளாதார சீர்குலைவை ஏற்படுத்தியுள்ளது.இதனால் பெரும் பொருளாதார மந்தநிலைக்குப் பின்னர் உலகளவில் மிகப்பெரிய மந்தநிலை ஏற்பட்டுள்ளது. இது விளையாட்டு,மத, அரசியல் மற்றும் கலாச்சார நிகழ்வுகளை ஒத்திவைக்க அல்லது ரத்து செய்ய வழிவகுத்தது.
# அச்சம் காரணமாக முகக்கவசம், கிருமிநாசினி உள்ளிட்ட பொருட்களை அதிக நபர்கள் வாங்கியதால் விநியோகப் பற்றாக்குறை ஏற்பட்டது."""

# indic2en_model.translate_paragraph(ta_paragraph, 'ta', 'en')

100%|██████████| 4/4 [00:00<00:00, 4304.06it/s]
/usr/local/lib/python3.7/dist-packages/fairseq/search.py:140: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beams_buf = indices_buf // vocab_size
/usr/local/lib/python3.7/dist-packages/fairseq/sequence_generator.py:666: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  unfin_idx = bbsz_idx /

'The pandemic has caused global social and economic disruption. This has led to the worlds largest recession since the Great Depression. This led to the postponement or cancellation of sporting, religious, political and cultural events. Due to this fear, there was a shortage of supply as most of the people purchased the items like masks, sanitizers etc.'